#  Submit geo opt for molecules, slabs and bulks

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
# General imports.
import ipywidgets as ipw
from IPython.display import clear_output

# AiiDA imports.
%load_ext aiida
%aiida
from aiida.plugins import  WorkflowFactory , DataFactory
from aiida.orm import  load_code

# AiiDAlab imports.
import aiidalab_widgets_base as awb

# Custom imports.
from widgets.build_slab import BuildSlab
from widgets.computational_resources import ProcessResourcesWidget, ResourcesEstimatorWidget
from widgets.inputs import InputDetails
from widgets.empa_viewer import EmpaStructureViewer
from widgets.ANALYZE_structure import StructureAnalyzer
#from widgets.import_cdxml import CdxmlUpload2GnrWidget



In [ ]:
Cp2kGeoOptWorkChain = WorkflowFactory('nanotech_empa.cp2k.geo_opt')

In [ ]:
# Structure selector.
empa_viewer = EmpaStructureViewer()
build_slab = BuildSlab(title='Build slab')
ipw.dlink((empa_viewer, 'details'), (build_slab, 'details'))
ipw.dlink((empa_viewer, 'structure'), (build_slab, 'molecule'))

structure_selector = awb.StructureManagerWidget(
    viewer=empa_viewer,
    importers=[
        awb.StructureUploadWidget(title="Import from computer"),
        awb.StructureBrowserWidget(title="AiiDA database"),
        awb.OptimadeQueryWidget(embedded=True),
        awb.SmilesWidget(title="From SMILES"),
        #CdxmlUpload2GnrWidget(title="CDXML"),
    ],
    editors = [
        awb.BasicStructureEditor(title="Edit structure"),
        build_slab
    ],
    storable=False, node_class='StructureData')
display(structure_selector)

# Code.
computational_resources = awb.ComputationalResourcesWidget(
            description="CP2K code:", default_calc_job_plugin="cp2k"
        )
resources = ProcessResourcesWidget()

input_details = InputDetails()
create_input = ipw.Button(description="Create Input")

# Resources

In [ ]:
ipw.dlink((empa_viewer, 'details'), (input_details, 'details'))
ipw.dlink((computational_resources, 'value'),(input_details, 'selected_code'))

def prepare_geo_opt():

    dft_params = {}
    sys_params = {}

    builder = Cp2kGeoOptWorkChain.get_builder()
    parameters = input_details.final_dictionary.copy()
    builder.code = load_code(computational_resources.value)
    builder.structure = structure_selector.structure_node
    builder.charge = Int(parameters['charge'])
    
    ase_geom =  structure_selector.structure        

    # spin guess
    mag_list = [ 0 for t in ase_geom ]
    if parameters['multiplicity']>0:
        for i in awb.utils.string_range_to_list(parameters['spin_u'])[0]:
            mag_list[i] = 1
        for i in awb.utils.string_range_to_list(parameters['spin_d'])[0]:
            mag_list[i] = -1 
        builder.multiplicity = Int(parameters['multiplicity'])
        
    builder.magnetization_per_site = List(mag_list)    
    builder.vdw = Bool(parameters['vdw_switch'])
    builder.fixed_atoms = Str(parameters['fixed_atoms'])
    builder.protocol = Str(parameters['protocol'])
    builder.options = {
            "max_wallclock_seconds": resources.walltime_seconds,
            "resources": {
                'num_machines': resources.nodes,
                'num_mpiprocs_per_machine': resources.tasks_per_node,
                'num_cores_per_mpiproc': resources.threads_per_task,
            },
        }
    
    builder.metadata.label = 'Cp2kGeoOptWorkChain'
    builder.metadata.description = parameters['description'] 

    return builder

In [ ]:
def after_submission(_=None):   
    structure_selector.value = None
    
btn_submit = awb.SubmitButtonWidget(Cp2kGeoOptWorkChain, 
                                inputs_generator=prepare_geo_opt
                               )

btn_submit.on_submitted(after_submission)

In [ ]:
output = ipw.Output()
def update_all(_=None):
    btn_submit.btn_submit.disabled=True 

def create(_=None):
    if computational_resources.value is None:
        msg = 'Please selecte a code first.'
        can_submit = False
    else:            
        can_submit, msg, details_dict = input_details.return_final_dictionary()
 
    with output:
        clear_output()
        if can_submit:
            print("Ready to submit a ",details_dict['workchain']," type of calculation" )
            if details_dict['workchain'] == 'Cp2kGeoOptWorkChain':
                btn_submit.btn_submit.disabled = not can_submit
                display(btn_submit)
        else:
            print(msg)

create_input.on_click(create)
input_details.observe(update_all, names='details')

In [ ]:
# Resources estimation.
resources_estimation = ResourcesEstimatorWidget()
resources_estimation.link_to_resources_widget(resources)
ipw.dlink((empa_viewer, 'details'), (resources_estimation, 'details'))
ipw.dlink((input_details, 'uks'), (resources_estimation, 'uks'))
_ = ipw.dlink((computational_resources, 'value'), (resources_estimation, 'selected_code'))

In [ ]:
display(ipw.VBox([input_details, resources, resources_estimation, computational_resources, create_input]), output)